In [12]:
import ecc
import util
import hashlib
from os import urandom
from binascii import hexlify
import bitcoin

In [3]:
# A commitment is a piece of information that bind me to some value without revealing it
# It is usually done with a one way function, or hash
commitment = hashlib.sha256(bytes(78)).digest()
print(hexlify(commitment))

b'5552748b5aeb500f57b3d1f4a56e4e9789198918c663e712314ea999026eb896'


In [4]:
# Now I can give the commitment to someone else and if I reveal the message later he can hash it again
# and verify that the hash is the same, meaning that I didn't change my mind

# But there's an issue with this naive commitment: if I know that the commited value is in a limited set,
# I can hash all the possible value until I find the right one.

# For example, if I know that the commitment is an int between 0 and 100, it is very easy to find 
for i in range(0, 200):
    if hashlib.sha256(bytes(i)).digest() == commitment:
        print(i)

78


In [5]:
# To avoid that problem, I can add another piece of random bytes to the value commited, so that you can't 
# easily find out but still verify if I give you the random bytes later
commitment = hashlib.sha256(bytes(2) + b'coucou').digest()
print(hexlify(commitment))

b'9dd8499cdf53b0c88e9952097aec2bc7f3786c78861fbcc62f05ed9cb8dedca2'


In [6]:
# This kind of commitment is not homomorphic, meaning that H(A) + H(B) != H(A+B)
# We can use modular arithmetic to achieve this.
# In this very simple example, I provide a commitment that have been computed with a generator g to the power 
# of the value I need to commit
# This is secured by the discrete log problem, meaning that given C, g, and N it is hard to find m
# Of course we need to use much larger values to be secure
N = 7
g = 3 # 3 is a generator for the group of integers mod N
m = 100
C = g**m % N
print(C)

4


In [7]:
# Now we have the same semantic security problem, meaning that the same m would always produce the same commitment
# We solve this by adding a second generator, h, to the power of some random value, and multiply our commitment
h = 5 # h is public
r = int.from_bytes(urandom(1), 'little') % N # r is secret and revealed only when I open the commitment
C = (g**m * h**r)
print(C)

2576887603660056655182305648828106363510537610005


In [11]:
# Now the interesting part: we can use the homomorphism of Pedersen Commitment to create commitments that can add up
# For example, imagine that Alice first commit to some value A
A = 23
r_A = int.from_bytes(urandom(1), 'little') % N
C_A = (g**A * h**r_A)

# She sends this first commitment to Bob, and some time later commit to another value B
B = 32
r_B = int.from_bytes(urandom(1), 'little') % N
C_B = (g**B * h**r_B)

# Bob now has 2 commitments to some values, and also a commitment to the sum of A and B
C_AB = C_A * C_B
assert C_AB == g**(A+B) * h**(r_A+r_B)

# Bob can now verify that this third commitment is really a commitment to the sum of A and B
# without ever knowing any value commited

# Another interesting consequence is that I can verify that the sum of some values is equal to some other
# This obviously will help when we need to prove that no bitcoins were created in a transaction
C = 40
r_C = int.from_bytes(urandom(1), 'little') % N
C_C = (g**C * h**r_C)

D = 15
r_D = r_A + r_B - r_C
C_D = (g**D * h**r_D)

C_CD = C_C * C_D
assert C_AB - C_CD == 0

In [10]:
# Another interesting property is that the commitment to the sum would work even with different A and B
# as long as A' + B' = AB
C_AB_prime = g**((A-3)+(B+3)) * h**(r_A+r_B)
assert C_AB == C_AB_prime

# And the same with the random value
C_AB_prime = g**((A)+(B)) * h**((r_A-2)+(r_B+2))
assert C_AB == C_AB_prime

In [14]:
# We can produce the same commitment with ECC
G = ecc.G # we use the generator used in Bitcoin
m = int.from_bytes(b'This is a commitment to some value', 'little')
C_A = m * G
print(C_A)

S256Point(27d615cd7789eac376bf4875f0c22e64839b38408eebfb01964cc47fc959c9b1, bcf2ee4a5e1749df08203337c12619a77871cda46679e8f99e26ee7919c165c6)


In [15]:
# We still need to find another generator point H
# in CT, they chosed to hash the x coordinate of G
to_hash = G.sec(False)
H_x = hashlib.sha256(to_hash).digest()
H_x = int(H_x, 16)

# We then need to find the y coordinate
H_y = pow(int(H_x*H_x*H_x + 7), int((ecc.P+1)//4), int(ecc.P))

H = ecc.S256Point(H_x)
print(H)

ValueError: invalid literal for int() with base 16: b'P\x92\x9bt\xc1\xa0IT\xb7\x8bK`5\xe9z^\x07\x8aZ\x0f(\xec\x96\xd5G\xbf\xee\x9a\xce\x80:\xc0'